In [1]:
import numpy as np
import pandas as pd
from neo4j import GraphDatabase, RoutingControl

In [2]:
DB_ULR = "neo4j://localhost:7687"
DB_USER = "neo4j"
DB_PASS = "test1234"
DB_NAME = "android"
DB_ENCRYPTED = False


In [3]:
driver = GraphDatabase.driver(DB_ULR, auth=(DB_USER, DB_PASS), encrypted=DB_ENCRYPTED)

In [ ]:
# Schema 
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (n:App) REQUIRE (n.id) IS NODE KEY").consume()
    )
    result = session.execute_write( lambda tx: 
        tx.run("CREATE CONSTRAINT IF NOT EXISTS FOR (n:Package) REQUIRE (n.name) IS NODE KEY").consume()
    )
    result = session.execute_write( lambda tx: 
        tx.run("CREATE INDEX IF NOT EXISTS FOR (p:App) ON (p.wcc)").consume()
    )

In [2]:
df = pd.read_csv("/Users/haklof/import/Android_Permission.csv").dropna()
df.head()

,App,Package,Category,Description,Rating,Number of ratings,Price,Related apps,Dangerous permissions count,Safe permissions count,...,Your personal information : read calendar events (D),Your personal information : read contact data (D),Your personal information : read sensitive log data (D),Your personal information : read user defined dictionary (D),Your personal information : retrieve system internal state (S),Your personal information : set alarm in alarm clock (S),Your personal information : write Browser's history and bookmarks (D),Your personal information : write contact data (D),Your personal information : write to user defined dictionary (S),Class
0,Canada Post Corporation,com.canadapost.android,Business,Canada Post Mobile App gives you access to som...,3.1,77,0.0,"{com.adaffix.pub.ca.android, com.kevinquan.gas...",7.0,1,...,0,1,0,0,0,0,0,1,0,0
1,Word Farm,com.realcasualgames.words,Brain & Puzzle,Speed and strategy combine in this exciting wo...,4.3,199,0.0,"{air.com.zubawing.FastWordLite, com.joybits.do...",3.0,2,...,0,0,0,0,0,0,0,0,0,0
2,Fortunes of War FREE,fortunesofwar.free,Cards & Casino,"Fortunes of War is a fast-paced, easy to learn...",4.1,243,0.0,"{com.kevinquan.condado, hu.monsta.pazaak, net....",1.0,1,...,0,0,0,0,0,0,0,0,0,0
3,Better Keyboard: Avatar Purple,com.cc.betterkeyboard.skins.avatarpurple,Libraries & Demo,Skin for Better Keyboard featuring a glossy fe...,3.6,2589,0.0,{eu.gdumoulin.betterandroid.skins.transparent....,0.0,0,...,0,0,0,0,0,0,0,0,0,0
5,Ms Claus Live Wallpaper,tmc.christmaslady.livewallpaper,Personalization,Ms Claus Live Wallpaper<p>Find more Free apps ...,4.5,16,0.0,"{tmc.christmassanta.livewallpaper, tmc.winterh...",3.0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.App.value_counts()

In [ ]:
apps = list()
packages = set()
app_installs = list()
app_links = list()
for idx, row in df.iterrows():
    app = { 
        'appId': idx,
        'name': row['App'],
        'description': row['Description'],
        'rating': row['Rating'],
        'numberOfRatings': row['Number of ratings'],
        'price': row['Price'],
        'dangerous': row['Dangerous permissions count'],
        'safe': row['Safe permissions count'],
        'rowNum': idx,
        'class': row['Class']
    }
    if row['Package'] != None:
        app_installs.append({'appId': idx, 'package': row['Package']})
    app_links.append({'appId': idx,  "relatedPackages": [ package.strip() for package in row['Related apps'][1:-1].split(',') ]})
    packages.add(row['Package'])
    [ packages.add(package.strip()) for package in row['Related apps'][1:-1].split(',') ]
    apps.append(app)        
print(app_links[0])

In [ ]:
# Create Package nodes
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run(
            """
            UNWIND $packages as package
            MERGE (a:Package{name:package})
            """,
            packages = list(packages)
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
# Create App nodes
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run(
            """
            UNWIND $apps as app
            MERGE (a:App{id:app.appId})
            SET a.name = app.name,
                a.description = app.description,
                a.rating = toFloat(app.rating),
                a.numberOfRatings = toInteger(app.numberOfRatings),
                a.price = toFloat(app.price),
                a.dangerous = toInteger(app.dangerous),
                a.safe = toInteger(app.safe),
                a.rowNum = app.rowNum,
                a.class = toInteger(app.class)
            """,
            apps = apps
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
# Create App - INSTALLS -> Package rels
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run(
            """
            UNWIND $app_installs as rel
            MATCH (a:App{id: rel.appId}), (p:Package{name: rel.package})
            MERGE (a)-[:INSTALLS]->(p)
            """,
            app_installs = app_installs
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
# Create App - LINKS -> Package rels
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run(
            """
            UNWIND $app_links as rel
            MATCH (a:App{id: rel.appId}), (p:Package)
            WHERE p.name in rel.relatedPackages
            MERGE (a)-[:LINKS]->(p)
            """,
            app_links = app_links
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
# Add label for Malware
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run(
            """
            MATCH (a:App) where a.class = 1
            set a:Malware
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
# Calculate dangerousness of app
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run(
            """
            MATCH (a:App)
            set a.dangerousness = a.dangerous/(a.safe + a.dangerous + 0.00001)
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
# Is an app more safe if other appls link what it installs?
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run(
            """
            MATCH (a:App)-[:INSTALLS]->(p:Package)
            set a.numberOfDependants = count { (p)<-[:INSTALLS]-() }
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
# Create graph projection 
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run("CALL gds.graph.drop('app-projection', false)").consume()
    )
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run(
            """
            CALL gds.graph.project.cypher('app-projection',
            'MATCH (a:App) return id(a) as id, 
                ["App"] as labels,
                a.class as class, 
                coalesce(a.numberOfRatings, 0) as numberOfRatings, 
                coalesce(a.rating,0.0) as rating, 
                coalesce(a.price,0.0) as price,
                coalesce(a.dangerous,0) as dangerous,
                coalesce(a.safe,0) as safe,
                a.dangerousness as dangerousness,
                a.numberOfDependants as numberOfDependants',
            'MATCH (a1:App)-[:LINKS|INSTALLS]->(p:Package)<-[:LINKS|INSTALLS]->(a2:App) 
            RETURN id(a1) as source, id(a2) as target, (a1.dangerousness+a2.dangerousness)/2 as risk
            ')
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
# Pagerank
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            call gds.pageRank.mutate('app-projection',{
                    maxIterations: 100,
                    mutateProperty: 'pageRank'
            })    
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
# WCC
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            call gds.wcc.mutate('app-projection',{
                    mutateProperty: 'wcc'
            })    
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
# Create embedding
with driver.session(database = DB_NAME) as session:
    result = session.write_transaction( lambda tx: 
        tx.run(
            """
            call gds.fastRP.mutate('app-projection',{
                embeddingDimension: 8, 
                relationshipWeightProperty: 'risk',
                iterationWeights: [0.0, 1.0, 0.8, 0.7],
                mutateProperty: 'embedding'
                })
                
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
## Todo: Upgrade to use ml pipeline
# with driver.session(database = DB_NAME) as session:
#     try: 
#         result = session.execute_write( lambda tx: 
#             tx.run("call gds.beta.model.drop('risk-model')").consume()
#         )
#     except:
#         print("Failed to remove model")
# with driver.session(database = DB_NAME) as session:
#     result = session.execute_write( lambda tx: 
#         # Todo: 'numberOfRatings' has NaN value?
#         tx.run(
#             """
#             call gds.ml.nodeClassification.train('app-projection',{
#                 nodeLabels: ['App'],
#                 modelName: 'risk-model',
#                 featureProperties: ['embedding', 'rating', 'price', 'dangerous', 'safe', 'dangerousness', 'numberOfDependants', 'pageRank', 'wcc'], 
#                 targetProperty: 'class', 
#                 metrics: ['F1_WEIGHTED', 'ACCURACY'], 
#                 holdoutFraction: 0.7, 
#                 validationFolds: 5, 
#                 randomSeed: 3,
#                 params: [
#                     {penalty: 0, minEpochs: 10, maxEpochs: 10000, tolerance: 0.00001, patience: 5},
#                     {penalty: 0.5, minEpochs: 10, maxEpochs: 10000, tolerance: 0.00001, patience: 5},
#                     {penalty: 1.0, minEpochs: 10, maxEpochs: 10000, tolerance: 0.00001, patience: 5}
#                     ]
#             }) yield modelInfo
#             return modelInfo
#             """
#         ).data()
#     )
#     print(json.dumps(result, indent=2))

In [ ]:
# Store feutures in graph
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run(
            """
            call gds.graph.nodeProperties.write('app-projection',['pageRank','wcc'])
            """
        ).data()
    )
    resultFrame = pd.DataFrame(result)
    display(resultFrame)

In [ ]:
# Drop projection
with driver.session(database = DB_NAME) as session:
    result = session.execute_write( lambda tx: 
        tx.run("CALL gds.graph.drop('app-projection', false)").consume()
    )

In [4]:
# Dump a sub section of the graph
df_community = driver.execute_query(
    ''' 
    match (a:App{wcc:15})
    with a,         
        [(a)-[:INSTALLS]->(p) | p.name ] [0] as Package,
        collect { match (a)-[:LINKS]->(p) with a, collect(p.name) as related return related }[0] as `Related apps`
    return  a.id as Id,
            a.name as App,
            a.description as Description,
            a.rating as Rating,
            a.numberOfRatings as `Number of ratings`,
            a.price as Price,
            a.dangerous as `Dangerous permissions count`,
            a.safe as `Safe permissions count`,
            a.class as Class,
            Package,
            `Related apps`
    ''',
    database_=DB_NAME,
    routing_=RoutingControl.READ,
    result_transformer_= lambda r: r.to_df()
)
df_community.head()

,Id,App,Description,Rating,Number of ratings,Price,Dangerous permissions count,Safe permissions count,Class,Package,Related apps
0,17,optionsXpress,OX Mobile puts live trading and the latest mar...,4.2,294,0.0,3,2,0,com.msf.opx,"[com.jw.EquityOptCalc, ek.options, jason.wei.a..."
1,32,OANDA fxTrade for Android,Get OANDA's tight spreads and forex trading ad...,4.0,757,0.0,1,1,1,com.oanda.fxtrade,"[com.yproky.fxCalendar, com.olsoft.mobitradeon..."
2,69,Xogee Trader,Xogee Limited provides software solutions to r...,4.4,10,0.0,2,0,1,com.xogee.trader,"[com.liteforex.trader, com.millenium.trader, c..."
3,78,My Car,Costs of running a car administration applicat...,4.6,104,0.0,2,1,1,cz.dorazil.jan.MyCar,"[cz.actum.cz.koop, org.polaris.bankomaty, net...."
4,93,Coors CU Mobile Banking,New App!<p>Coors Credit Union Mobile Banking a...,4.5,8,0.0,6,0,1,com.softek.ofxclmobile.coorscu,"[com.sfcu.org, com.softek.ofxclmobile.fibergla..."


In [5]:
df_community.to_csv('apps.csv', index=False)